In [1]:
from bs4 import BeautifulSoup
import webbrowser
import requests
import pandas as pd
import cv2 
import urllib.request as urli
import os 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
import pickle
import time 



options = Options()
options.add_argument("--headless")

# Load listings

In [2]:
url_listing = "http://data.insideairbnb.com/ireland/leinster/dublin/2021-11-07/data/listings.csv.gz"
listings = pd.read_csv(url_listing)
prefix = "https://airbnb.de"
suffix = "/photos"
urls = listings["listing_url"]
ids = listings["id"]


In [3]:
webbrowser.open(urls[0])

True

In [3]:
id_dict = dict()
for id, url in tqdm(zip(ids, urls), total = len(ids)):    
    id_list = []
    try:
        driver = webdriver.Chrome("/Users/dmnk/Documents/Dubair_Trash/chromedriver 2", options = options)
        
        driver.get(url+suffix)

        body = driver.find_element_by_tag_name("body")
        for i in range(40):
            body.send_keys(Keys.TAB)
        
        time.sleep(1)

        for i in range(40):
            body.send_keys(Keys.TAB)            

        html = driver.page_source
        html_content = BeautifulSoup(html, "html.parser")
        gallery_items = html_content.findAll("img", {"class": "_6tbg2q"})
    except:
        continue
    for i, item in enumerate(gallery_items[5:]):
        try:
            img_link = item.get("src")
            img_title = item.get("alt")
            path_name = "/Volumes/TOSHIBA EXT/images_new/"+str(id)+"_"+str(i)+".png"
            if os.path.exists(path_name):
                id_list.append(img_title)
                continue
            else:
                urli.urlretrieve(img_link, path_name)
                id_list.append(img_title)
            
        except:
            print("ups - fail")
            continue

    id_dict[id] = id_list
meta_images = open("/Volumes/TOSHIBA EXT/images/data.pkl", "wb")
pickle.dump(id_dict, meta_images)
meta_images.close

  0%|          | 0/6976 [00:00<?, ?it/s]/var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_88378/3013026069.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/dmnk/Documents/Dubair_Trash/chromedriver 2", options = options)
/var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_88378/3013026069.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  body = driver.find_element_by_tag_name("body")
 23%|██▎       | 1626/6976 [2:19:50<9:10:03,  6.17s/it]  

ups - fail


 24%|██▎       | 1644/6976 [2:21:21<8:15:35,  5.58s/it]

ups - fail


In [11]:
import json

with open('labels_raw.json', 'w') as fp:
    json.dump(id_dict, fp)

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


OSError: [Errno 28] No space left on device: 'labels_raw.json'

Resize images to same shape

In [ ]:
resized_path = "data/images_resized/"

dir = os.listdir("images")

img_names = pd.Series(dir).str.split(".").str[0].values

for img_dir, resized_img_dir in tqdm(zip(dir, img_names), total = len(dir)):
  img_dir_tmp = "images/"+ img_dir
  img_tmp = cv2.imread(img_dir_tmp)
  img_tmp = cv2.resize(img_tmp, dsize=(256, 256))
  path_tmp = resized_path + resized_img_dir + ".png"
  cv2.imwrite(path_tmp, img_tmp)